In [ ]:
import requests
import re
from bs4 import BeautifulSoup

La siguiente función realiza un web scraping de la web de codigos postales tomando como argumentos provincia, localidad, calle y altura de la calle
y devuelve como lista los datos de localización con el cpa correspondiente. mi propuesta es desplegar estas unciones en una api y a partir de ella obtener la información de la forma que se requiera. se tuvo en cuenta todas las variables del scraping como localidades sin calles, localidades con calles y alturas pares e impares, localidades con calles pero sin alturas, todo puede ser scrapeado y regresa el cpa exacto.

In [267]:
def obtener_info_postal(provincia, localidad="", calle="", altura=""):
    # Cambiar los espacios por guiones
    provincia = provincia.replace(" ", "-")
    localidad = localidad.replace(" ", "-")
    calle = calle.replace(" ", "-")  


    #Construir la URL de la provincia.    
    url = f'https://codigo-postal.co/argentina/{provincia}/'

    # Realizar la solicitud GET a la página web
    response = requests.get(url)

    # Crear el objeto BeautifulSoup para analizar el contenido HTML
    soup = BeautifulSoup(response.content, "html.parser")

    # Encontrar el elemento <p> y obtener el texto
    p_element = soup.find("p")
    cpa_text = p_element.text.strip()
    punto_index = cpa_text.index('.')  # Encuentra el índice del punto
    texto_cortado = cpa_text[:punto_index + 1]

    cantidad_cpa = re.search(r'\d+', texto_cortado).group()
    cantidad_cpa = int(cantidad_cpa)

    #Verificar si existen CPA para la provincia
    if cantidad_cpa != 0:

        if not localidad:
            return f"La provincia de {provincia} posee {cantidad_cpa} códigos postales diferentes. Elige una de las localidades"
        
        else:
             # Construir la URL
            url = f'https://codigo-postal.co/argentina/{provincia}/{localidad}/'

            # Realizar la solicitud GET a la página web
            response = requests.get(url)

            # Crear el objeto BeautifulSoup para analizar el contenido HTML
            soup = BeautifulSoup(response.content, "html.parser")

            # Encontrar el elemento <div class="question"> y obtener el texto del primer <p> dentro de él
            div_element = soup.find("div", class_="question")
            p_element = div_element.find('p')
            cpa_text = p_element.text.strip()

            # Crear una variable para verifficar que la localidad tiene mas de un CPA y requiere ingresar la calle
            texto = "más de uno"
            
            # Ejecutar si la localidad tiene mas de un CPA
            if texto in cpa_text:

                texto2 = "Existe más de uno CPA para en . "
                
                if texto2 not in cpa_text:                                                          
                    # Ejecutar si no se indico la calle de la localidad
                    if not calle:
                        return cpa_text
                    
                    # Ejecutar si se indico la calle de la localidad
                    else:
                        # Construir la URL
                        url = f'https://codigo-postal.co/argentina/{provincia}/{localidad}/{calle}/'

                        # Realizar la solicitud GET a la página web
                        response = requests.get(url)

                        # Crear un objeto BeautifulSoup con el contenido HTML recibido
                        soup = BeautifulSoup(response.text, 'html.parser')

                        # Encontrar la tabla por su ID
                        table = soup.find('table', {'id': 'id_table_cpa'})

                        if table is None:
                            div_element = soup.find("div", class_="col-md-8")
                            strong_element = div_element.find("strong")

                            if strong_element is None:
                                return f"No hemos podido encontrar la calle {calle}"
                        
                            else:                            
                                cpa = strong_element.text.strip()
                                return (provincia,localidad,calle,altura,cpa)
                                            
                        else:
                            # Obtener las filas de la tabla (<tbody>)
                            rows = table.find('tbody').find_all('tr')

                            # Obtener el rango mínimo y máximo de los registros en la tabla
                            registros = []
                            for row in rows:
                                cells = row.find_all('td')
                                desde = int(cells[1].text.strip())
                                hasta = int(cells[2].text.strip())
                                registros.append((desde, hasta))
                            
                            # Obtener el rango mínimo y máximo de los registros en la tabla
                            min_desde = min(registros, key=lambda x: x[0])[0]
                            max_hasta = max(registros, key=lambda x: x[1])[1]

                            # Verificar si el valor de la altura se encuentra dentro del rango de los registros
                            if altura < min_desde or altura > max_hasta:
                                return f"La altura de calle n° {altura} no se encuantra en el rango de calles de la calle {calle}, su altura va desde {min_desde} hasta {max_hasta}"
                        
                            # Filtrar los registros en función de la altura, rango y aplica a
                            registros_filtrados = []
                            for row in rows:
                                cells = row.find_all('td')
                                desde = int(cells[1].text.strip())
                                hasta = int(cells[2].text.strip())
                            
                                if desde <= altura <= hasta and ((altura % 2 == 0 and cells[3].text.strip() == 'números pares') or (altura % 2 != 0 and cells[3].text.strip() == 'números impares')):
                                    registros_filtrados.append(cells[-1].text.strip())  # Agregar el valor de la última columna "CPA" al registro filtrado

                            # Verificar si no se encontraron registros correspondientes al rango de altura y aplica a
                            if not registros_filtrados:
                                if altura % 2 == 0:
                                    return f"la calle {calle} solo tiene CPA a la altura de numeros de altura impares"
                                else:
                                    return f"la calle {calle} solo tiene CPA a la altura de numeros de altura pares"

                            # Devolver el primer valor de la lista de registros filtrados
                            return (provincia,localidad,calle,altura,registros_filtrados[0])
                
                elif texto2 in cpa_text:
                    return f"no hemos podido encontrar el codigo postal de {localidad} en la provincia {provincia}"

                        
            else:
                strong_element = div_element.find("strong")
                cpa = strong_element.text.strip()
                return (provincia,localidad,calle,altura,cpa)    
            
    else:
        return f"no hemos podido encontrar la provincia {provincia}"


In [268]:
provincia = 'capital federal'
localidad = 'ciudad autonoma buenos aires'
calle = '24 de noviembre'
altura = 2200

cpa = obtener_info_postal(provincia, localidad,calle,altura)
print (cpa)

('capital-federal', 'ciudad-autonoma-buenos-aires', '24-de-noviembre', 2200, 'C1242AAT')


Esta funcion permite consultar un CPA y extarer la localización de ese cpa, al igual que la anterior se puede desplegar en una API tipo REST como consulta get y va a devolver la localizacion de ese CPA.

In [316]:
def obtener_localaizacion (cpa):
    
    url = f'https://codigo-postal.co/argentina/cpa/{cpa}/'

    # Realizar la solicitud GET a la página web
    response = requests.get(url)

    # Crear el objeto BeautifulSoup para analizar el contenido HTML
    soup = BeautifulSoup(response.content, "html.parser")

    # Encontrar el elemento <div class="question"> y obtener el texto del primer <p> dentro de él
    div_element = soup.find("div", class_="col-md-8")
    p_element = div_element.find("p")
    text = p_element.text.strip()

    # Extraer la provincia
    provincia_match = re.search(r"provincia\s([\w\s]+),", text)
    provincia = provincia_match.group(1).strip() if provincia_match else None

    # Extraer la localidad
    localidad_match = re.search(r"en\s([\w\s-]+),", text)
    localidad = localidad_match.group(1).strip().replace("-", " ") if localidad_match else None

    # Extraer la calle
    calle_match = re.search(r"Calle\s([\w\s]+)\sa", text)
    calle = calle_match.group(1).strip() if calle_match else None

    # Eliminar la parte adicional de la calle
    if calle:
        calle = re.sub(r"a todos los números.+?$", "", calle)

    # Extraer el rango de números
    numeros_match = re.search(r"los\snúmeros\s(\d+)\sa\s(\d+)", text)
    inicio = int(numeros_match.group(1)) if numeros_match else None
    fin = int(numeros_match.group(2)) if numeros_match else None

    # Extraer el código postal
    cp_match = re.search(r"CPA\s([A-Z0-9]+)", text)
    cp = cp_match.group(1) if cp_match else None

    # Crear la lista con los valores extraídos
    resultado = [provincia, localidad, calle,inicio, fin, cp]

    # Imprimir la lista resultante
    return resultado

In [318]:
localizacion = obtener_localaizacion ("C1242AAT")
print (localizacion)

['Capital Federal', 'Ciudad Autonoma Buenos Aires', '24 de Noviembre ', 2102, 2200, 'C1242AAT']
